In [1]:
# start SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("project").getOrCreate()

# 1. STORIES

In [6]:
# read in the dataset stories
data_df = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://xinyuwang/finalproject/story.csv')

In [7]:
 data_df.show()

+-------+------------+-----+------------+--------------------+--------------------+--------------------+--------------------+-------+----+-----------+------------+
|     id|          by|score|        time|             time_ts|               title|                 url|                text|deleted|dead|descendants|      author|
+-------+------------+-----+------------+--------------------+--------------------+--------------------+--------------------+-------+----+-----------+------------+
|6940813|   sarath237|  0.0|1387536270.0|2013-12-20 10:44:...| Sheryl Brindo Ho...|http://www.youtub...| Sheryl Brindo Ho...|   null|True|       null|   sarath237|
|6991401|123123321321|  0.0|1388508751.0|2013-12-31 16:52:...|Are you people al...|                null|They&#x27;re pret...|   null|True|       null|123123321321|
|1531556|         ssn|  0.0|1279617234.0|2010-07-20 09:13:...|New UI for Google...|http://googlesyst...|Again following o...|   null|null|        0.0|         ssn|
|5012398|       

In [8]:
data_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- by: string (nullable = true)
 |-- score: string (nullable = true)
 |-- time: string (nullable = true)
 |-- time_ts: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- dead: string (nullable = true)
 |-- descendants: string (nullable = true)
 |-- author: string (nullable = true)



In [9]:
data_df.count()

2069464

In [10]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

story_df = data_df.select(data_df['id'].cast(IntegerType()),
                     data_df['score'].cast(IntegerType()),
                     #data_df["time"],
                     data_df['time_ts'].cast('timestamp'),
                     data_df['title'],
                     #data_df['type'],
                     data_df['url'],
                     data_df['text'],
                     #data_df['parent'].cast(IntegerType()),
                     #data_df['deleted'],
                     #data_df['dead'],
                     data_df['descendants'].cast(IntegerType()),
                     #data_df['id'].cast(IntegerType()),
                     data_df['author']
                             )


In [11]:
story_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- score: integer (nullable = true)
 |-- time_ts: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text: string (nullable = true)
 |-- descendants: integer (nullable = true)
 |-- author: string (nullable = true)



In [12]:
story_df.show(10)

+-------+-----+-------------------+--------------------+--------------------+--------------------+-----------+------------+
|     id|score|            time_ts|               title|                 url|                text|descendants|      author|
+-------+-----+-------------------+--------------------+--------------------+--------------------+-----------+------------+
|6940813|    0|2013-12-20 10:44:30| Sheryl Brindo Ho...|http://www.youtub...| Sheryl Brindo Ho...|       null|   sarath237|
|6991401|    0|2013-12-31 16:52:31|Are you people al...|                null|They&#x27;re pret...|       null|123123321321|
|1531556|    0|2010-07-20 09:13:54|New UI for Google...|http://googlesyst...|Again following o...|          0|         ssn|
|5012398|    0|2013-01-05 12:11:17|Historic website ...|http://webscrapin...|Python script to ...|          0|        hoju|
|7214182|    0|2014-05-31 18:42:20|         Placeholder|                null|       Mind the gap.|          0|       kogir|
|1187303

In [13]:
# create temp table
story_df.createOrReplaceTempView('ini_story')

In [14]:
story_df.select("id").distinct().count()

1959840

In [15]:
story_df.select("title").distinct().count()

1759584

In [16]:
story_df.select("time_ts").distinct().count()

1925929

In [17]:
clean_story_df = spark.sql('select * from ini_story where time_ts is not NULL and title is not NULL and author is not NULL and score is not NULL')
clean_story_df.createOrReplaceTempView('story')

In [49]:
url_df = spark.sql('select * from ini_story where url is not NULL and time_ts is not NULL and title is not NULL and author is not NULL and score is not NULL')

In [51]:
url_df.createOrReplaceTempView('url')

In [136]:
pattern = re.compile(r'([a-z]+).(com|net)')

In [137]:
re.findall(pattern, 'http://www.efficientsoftware.net/')

[('efficientsoftware', 'net')]

In [150]:
sqlStatement = """
SELECT id, regexp_extract(url, '([a-z]+).(com|net|co|cn|org|ru|de|br|uk|pl|ir|it|in|fr|au|jp|info)', 1) as web
FROM url 
"""

In [151]:
url_df1 = spark.sql(sqlStatement)

In [152]:
url_df1.show()

+-------+-----------------+
|     id|              web|
+-------+-----------------+
|6940813|          youtube|
|1531556|         blogspot|
|5012398|      webscraping|
|2606988|         blogspot|
|4542754|     windowsphone|
|4824675|       abiquolabs|
|1120285|       datingsite|
|5221577|        nobugware|
|4827038|   allelectronics|
|4481018|         blogspot|
|7061983|efficientsoftware|
|1215015|       technobuzz|
|1505655|         zohmuomo|
|4673231| gumrukmusaviritr|
|1503352|       roadtickle|
|1540745|      kronikmedia|
|1370228|       fastessays|
|5917267|     sextoysbrand|
|5969633|    matthewaperry|
| 124722|             blog|
+-------+-----------------+
only showing top 20 rows



In [153]:
url_df1.createOrReplaceTempView('url2')

In [157]:
sqlStatement = """
SELECT web
FROM url2
WHERE web != ''
"""
url_df2 = spark.sql(sqlStatement)

In [158]:
url_df2.count()

1657898

In [161]:
sqlStatement = """
SELECT COUNT(id) AS num, web
FROM url2
WHERE web != '' and web != 'www'
GROUP BY web
ORDER BY num DESC
"""
url_df3 = spark.sql(sqlStatement)

In [162]:
url_df3.show()

+-----+---------------+
|  num|            web|
+-----+---------------+
|36296|         github|
|30887|     techcrunch|
|30856|        youtube|
|28325|       blogspot|
|27659|        nytimes|
|18549|         medium|
|17192|         google|
|14755|            bbc|
|13796|    arstechnica|
|13624|          wired|
|12499|      wordpress|
| 8582|      wikipedia|
| 8156|businessinsider|
| 7434|         forbes|
| 7276|             on|
| 7183|           blog|
| 7110|       mashable|
| 7068|    venturebeat|
| 6812|     thenextweb|
| 6749|       theverge|
+-----+---------------+
only showing top 20 rows



In [136]:
# story distribution in terms of time
# What is the trend of story numbers from 2006-2016 in Hacker News?

sqlStatement = """
SELECT COUNT(id) AS story_num, 
year(time_ts) as year
FROM story
GROUP BY year
ORDER BY year
"""
story_time_df1 = spark.sql(sqlStatement)

In [137]:
story_time_df1.show()

+---------+----+
|story_num|year|
+---------+----+
|       50|2006|
|    21948|2007|
|    68877|2008|
|   109468|2009|
|   175246|2010|
|   285433|2011|
|   305361|2012|
|   303813|2013|
|   287287|2014|
|   250348|2015|
+---------+----+



In [138]:
# story distribution in terms of time
# What is the most popular/active time during a day in pubulishing a story?

sqlStatement = """
SELECT COUNT(id) AS story_num, 
hour(time_ts) as hour
FROM story
GROUP BY hour
ORDER BY story_num DESC
"""
story_time_df2 = spark.sql(sqlStatement)

In [139]:
story_time_df2.show(24)

+---------+----+
|story_num|hour|
+---------+----+
|   118330|  16|
|   116519|  17|
|   114846|  15|
|   111187|  18|
|   107289|  14|
|   101710|  19|
|    95143|  20|
|    92281|  13|
|    87559|  21|
|    75879|  12|
|    75403|  22|
|    64806|  23|
|    62944|  11|
|    58054|   0|
|    55560|  10|
|    54969|   1|
|    54117|   9|
|    53419|   2|
|    52016|   3|
|    51697|   6|
|    51264|   8|
|    51254|   4|
|    50800|   7|
|    50785|   5|
+---------+----+



In [93]:
# score information
# what is the average score of those story?
spark.sql("select min(score),max(score),avg(score) from story").collect()

[Row(min(score)=0, max(score)=75248, avg(score)=10.447452950680841)]

In [94]:
# author/user information 
# decending by total_score 
sqlStatement = """SELECT SUM(score) as total_score,author,
SUM(descendants) as total_decendants,
count(id) as total_stories
FROM story
GROUP BY author
ORDER BY total_score DESC
LIMIT 20
"""
user_df = spark.sql(sqlStatement)
#user_df.createOrReplaceTempView("user")

In [95]:
user_df.show(10)

+-----------+-----------+----------------+-------------+
|total_score|     author|total_decendants|total_stories|
+-----------+-----------+----------------+-------------+
|      76851|ColinWright|           30700|         3726|
|      75248|          0|         1732237|            0|
|      64156| shawndumas|           25926|         6585|
|      59958|    llambda|           20324|         2595|
|      56798|      fogus|           21136|         2412|
|      53452|      danso|           22792|         2610|
|      52701|       cwan|           24067|         7059|
|      51836|        luu|           19713|         2265|
|      49003|  ssclafani|           24026|         1324|
|      41601|      evo_9|           16215|         5635|
+-----------+-----------+----------------+-------------+
only showing top 10 rows



In [96]:
# author/user information 
# decending by total_stories 
sqlStatement = """SELECT SUM(score) as total_score, author,
SUM(descendants) as total_decendants,
count(id) as total_stories
FROM story
GROUP BY author
ORDER BY total_stories DESC
LIMIT 20
"""
user_df2 = spark.sql(sqlStatement)

In [97]:
user_df2.show(10)

+-----------+-----------+----------------+-------------+
|total_score|     author|total_decendants|total_stories|
+-----------+-----------+----------------+-------------+
|      52701|       cwan|           24067|         7059|
|      64156| shawndumas|           25926|         6585|
|      41601|      evo_9|           16215|         5635|
|      29477|      nickb|           11804|         4303|
|      26431|   iProject|           11759|         4262|
|      28454|   bootload|           11351|         4158|
|      29598|     edw519|           13726|         3823|
|      76851|ColinWright|           30700|         3726|
|      29806|     nreece|           12554|         3713|
|      36511| tokenadult|           20190|         3634|
+-----------+-----------+----------------+-------------+
only showing top 10 rows



In [98]:
# author/user information 
# decending by avg_score 
sqlStatement = """SELECT SUM(score) as total_score, author,
SUM(descendants) as total_decendants, SUM(score)/count(id) as avg_score,
count(id) as total_stories
FROM story
GROUP BY author
ORDER BY avg_score DESC
LIMIT 20
"""
user_df3 = spark.sql(sqlStatement)

In [99]:
user_df3.show(10)

+-----------+---------------+----------------+---------+-------------+
|total_score|         author|total_decendants|avg_score|total_stories|
+-----------+---------------+----------------+---------+-------------+
|       1543|    realfuncode|             526|   1543.0|            1|
|       2905|     frederfred|             412|   1452.5|            2|
|       1344| themanthatfell|             407|   1344.0|            1|
|       1282|          rcina|             249|   1282.0|            1|
|       1257|         kvargs|             558|   1257.0|            1|
|       1248|        mmebane|             267|   1248.0|            1|
|       1227|FlemishBeeCycle|             444|   1227.0|            1|
|       1172|     hannahmitt|             136|   1172.0|            1|
|       1125|  afraidofadria|             985|   1125.0|            1|
|       4354|   patricktomas|             385|   1088.5|            4|
+-----------+---------------+----------------+---------+-------------+
only s

In [100]:
# if there is bias on YC
sqlStatement = """
SELECT score, title,id
FROM `story`
WHERE title like "%Y Combinator%" or title like "%YCombinator%" or title like "%ycombinator%" or title like "%y combinator%"
ORDER BY score  DESC
"""
YC_df = spark.sql(sqlStatement)

In [101]:
YC_df.distinct().show()

+-----+--------------------+--------+
|score|               title|      id|
+-----+--------------------+--------+
| 1065|Y Combinator is f...| 5059806|
|  841|Y Combinator has ...| 8033322|
|  705|Meet Watsi, Y Com...| 5117385|
|  687|New: Apply to Y C...| 3700712|
|  634|I Am Sam Altman, ...| 9238839|
|  589|How I Got Kicked ...| 2208155|
|  550|Benefits matter, ...| 5409273|
|  549|What Happens At Y...| 1733236|
|  542|How Y Combinator ...| 3711008|
|  506|Y Combinator Numbers| 2608440|
|  432|New Y Combinator ...| 7972138|
|  425|Yuri Milner, SV A...| 2154706|
|  380|Y Combinator And ...| 8178450|
|  379|How I Crashed and...| 8867335|
|  368|Early Photos of Y...| 2942958|
|  366|Investment Firm Y...| 3492711|
|  344|Y Combinator anno...| 1898432|
|  342|Offer HN now at n...| 1840060|
|  337|A note I sent to ...| 4273460|
|  336|I am Sam Altman, ...|10360911|
+-----+--------------------+--------+
only showing top 20 rows



In [102]:
# stories of major companies over the year
# apple
sqlStatement = """
SELECT count(*) as total_stories, SUM(score) as total_score,
year(time_ts) as year
FROM story
WHERE title like "%apple%" or title like "%APPLE" or title like "%Apple"
GROUP BY year
ORDER BY year
"""
Comp_df1 = spark.sql(sqlStatement)

In [103]:
Comp_df1.distinct().show()

+-------------+-----------+----+
|total_stories|total_score|year|
+-------------+-----------+----+
|            1|          7|2006|
|           28|        109|2007|
|           43|        324|2008|
|           86|        780|2009|
|          194|       1964|2010|
|          348|       5316|2011|
|          382|       4883|2012|
|          280|       2345|2013|
|          203|       2302|2014|
|          168|       3672|2015|
+-------------+-----------+----+



In [104]:
# stories of major companies over the year
# google
sqlStatement = """
SELECT count(*) as total_stories, SUM(score) as total_score,
year(time_ts) as year
FROM story
WHERE title like "%google%" or title like "%GOOGLE" or title like "%Google"
GROUP BY year
ORDER BY year
"""
Comp_df2 = spark.sql(sqlStatement)

In [105]:
Comp_df2.distinct().show()

+-------------+-----------+----+
|total_stories|total_score|year|
+-------------+-----------+----+
|            1|          1|2006|
|          102|        514|2007|
|          190|       1192|2008|
|          428|       3376|2009|
|          553|       6938|2010|
|          506|       9489|2011|
|          461|       7592|2012|
|          478|      10844|2013|
|          432|       7835|2014|
|          369|       9247|2015|
+-------------+-----------+----+



In [106]:
# stories of major companies over the year
# uber
sqlStatement = """
SELECT count(*) as total_stories, SUM(score) as total_score,
year(time_ts) as year
FROM story
WHERE title like "%uber%" or title like "%UBER" or title like "%Uber"
GROUP BY year
ORDER BY year
"""
Comp_df3 = spark.sql(sqlStatement)

In [107]:
Comp_df3.distinct().show()

+-------------+-----------+----+
|total_stories|total_score|year|
+-------------+-----------+----+
|            3|          3|2007|
|           22|        202|2008|
|           23|        957|2009|
|           59|       1110|2010|
|           69|        966|2011|
|           58|        874|2012|
|           67|        523|2013|
|          244|       2743|2014|
|          302|       3301|2015|
+-------------+-----------+----+



# 2. COMMENTS

In [2]:
# read in the dataset comments
data_df2 = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load('s3://chingsez/Final/comments.csv')

In [3]:
data_df2.show()

+--------------------+-------+------+----------+--------------------+--------------------+--------------------+--------------------+-------+-------+
|                  id|     by|author|      time|             time_ts|                text|              parent|             deleted|   dead|ranking|
+--------------------+-------+------+----------+--------------------+--------------------+--------------------+--------------------+-------+-------+
|             2701393|     5l|    5l|1309184881|2011-06-27 14:28:...|And the glazier w...|             2701243|                null|   null|      0|
|             5811403|     99|    99|1370234048|2013-06-03 04:34:...|Does canada have ...|             5804452|                null|   null|      0|
|               21623|     AF|    AF|1178992400|2007-05-12 17:53:...|"Speaking of Rail...|               21611|                null|   null|      0|
|            10159727|     EA|    EA|1441206574|2015-09-02 15:09:...|Humans and large ...|            1015

In [64]:
data_df2.printSchema()

root
 |-- id: string (nullable = true)
 |-- by: string (nullable = true)
 |-- author: string (nullable = true)
 |-- time: string (nullable = true)
 |-- time_ts: string (nullable = true)
 |-- text: string (nullable = true)
 |-- parent: string (nullable = true)
 |-- deleted: string (nullable = true)
 |-- dead: string (nullable = true)
 |-- ranking: string (nullable = true)



In [14]:
data_df2.count()

9796725

In [15]:
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

comment_df = data_df2.select(
                     data_df2['id'].cast(IntegerType()),                    
                     data_df2['time_ts'].cast('timestamp'),                  
                     data_df2['text'],
                     data_df2['parent'].cast(IntegerType()),
                     #data_df2['deleted'],
                     #data_df2['dead'],
                     data_df2['author'],
                     data_df2['ranking'].cast(IntegerType())

                             )

In [16]:
comment_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- time_ts: timestamp (nullable = true)
 |-- text: string (nullable = true)
 |-- parent: integer (nullable = true)
 |-- author: string (nullable = true)
 |-- ranking: integer (nullable = true)



In [17]:
comment_df.show(10)

+--------+-------------------+--------------------+--------+------+-------+
|      id|            time_ts|                text|  parent|author|ranking|
+--------+-------------------+--------------------+--------+------+-------+
| 2701393|2011-06-27 14:28:01|And the glazier w...| 2701243|    5l|      0|
| 5811403|2013-06-03 04:34:08|Does canada have ...| 5804452|    99|      0|
|   21623|2007-05-12 17:53:20|"Speaking of Rail...|   21611|    AF|      0|
|10159727|2015-09-02 15:09:34|Humans and large ...|10159396|    EA|      0|
| 2988424|2011-09-12 18:53:00|I must say I reac...| 2988179|    Iv|      0|
| 3867418|2012-04-20 11:39:44|&#62; There's a w...| 3867404|    Iv|      0|
| 3925617|2012-05-03 20:26:05|I'm also in this ...| 3924840|    Iv|      0|
| 3107534|2011-10-13 15:34:04|how do you run un...|    null|    Iv|   null|
|    null|               null|                null|    null|  null|   null|
| 8409259|2014-10-04 11:20:47|Polio is not exte...| 8409226|    Iv|      0|
+--------+--

In [18]:
comment_df.createOrReplaceTempView('ini_comment')

In [19]:
comment_df.select("id").distinct().count()

8399564

In [20]:
clean_comment_df = spark.sql('select * from ini_comment where id is not NULL and time_ts is not NULL and text is not NULL and author is not NULL and ranking is not NULL')
clean_comment_df.createOrReplaceTempView('comment')

In [21]:
clean_comment_df.count()

6986995

In [22]:
clean_comment_df.show(10)

+--------+-------------------+--------------------+--------+------+-------+
|      id|            time_ts|                text|  parent|author|ranking|
+--------+-------------------+--------------------+--------+------+-------+
| 2701393|2011-06-27 14:28:01|And the glazier w...| 2701243|    5l|      0|
| 5811403|2013-06-03 04:34:08|Does canada have ...| 5804452|    99|      0|
|   21623|2007-05-12 17:53:20|"Speaking of Rail...|   21611|    AF|      0|
|10159727|2015-09-02 15:09:34|Humans and large ...|10159396|    EA|      0|
| 2988424|2011-09-12 18:53:00|I must say I reac...| 2988179|    Iv|      0|
| 3867418|2012-04-20 11:39:44|&#62; There's a w...| 3867404|    Iv|      0|
| 3925617|2012-05-03 20:26:05|I'm also in this ...| 3924840|    Iv|      0|
| 8409259|2014-10-04 11:20:47|Polio is not exte...| 8409226|    Iv|      0|
|   50570|2007-09-05 17:04:05|It was a risky jo...|   50556|    Jd|      0|
| 2600618|2011-05-30 22:34:14|"Looks good, ther...| 2600609|    Jd|      0|
+--------+--

In [23]:
# comment author/user information 
# who are the most contributive authors in Hacker News ?
## futhur to be done——can be joined with the most contirbutive authors in story

sqlStatement = """SELECT author,
        COUNT(id) AS total_comments
        From comment
        GROUP BY author
        ORDER BY total_comments DESC
        LIMIT 20
"""
comment_user_df = spark.sql(sqlStatement)

In [24]:
# show top 10
comment_user_df.show(10)

+------------+--------------+
|      author|total_comments|
+------------+--------------+
|     tptacek|         28605|
|    jacquesm|         19845|
|       DanBC|         10992|
|    jrockway|         10679|
|   anigbrowl|         10490|
|dragonwriter|         10203|
|         eru|          9979|
|     rbanffy|          9634|
|       sp332|          9547|
|     rayiner|          9403|
+------------+--------------+
only showing top 10 rows



In [25]:
# comment distribution in terms of time
# What is the trend of comment numbers from 2006-2016 in Hacker News?

sqlStatement = """
SELECT COUNT(id) AS comment_num, 
year(time_ts) as year
FROM comment
GROUP BY year
ORDER BY year
"""
comment_time_df2 = spark.sql(sqlStatement)

In [26]:
comment_time_df2.show()

+-----------+----+
|comment_num|year|
+-----------+----+
|         10|2006|
|      50982|2007|
|     195678|2008|
|     382490|2009|
|     658846|2010|
|     816999|2011|
|     975306|2012|
|    1415381|2013|
|    1348324|2014|
|    1142979|2015|
+-----------+----+



In [27]:
# comment distribution in terms of time
# What is the most popular/active time during a day in pubulishing a comment?

sqlStatement = """
SELECT COUNT(id) AS comment_num, 
hour(time_ts) as hour
FROM comment
GROUP BY hour
ORDER BY comment_num DESC
"""
comment_time_df = spark.sql(sqlStatement)

In [28]:
comment_time_df.show(24)

+-----------+----+
|comment_num|hour|
+-----------+----+
|     451259|  17|
|     450094|  18|
|     437369|  16|
|     433640|  19|
|     421504|  20|
|     413518|  15|
|     398429|  21|
|     365798|  14|
|     353708|  22|
|     313262|  23|
|     295136|  13|
|     276703|   0|
|     251256|   1|
|     236036|   2|
|     224956|   3|
|     224367|  12|
|     210740|   4|
|     192913|   5|
|     180795|   6|
|     180141|  11|
|     172147|   7|
|     170884|   8|
|     168104|   9|
|     164236|  10|
+-----------+----+



In [29]:
# top(the hottest) comments with the most follow-up comment 
sqlStatement = """
SELECT COUNT(id) AS followup_num, 
parent
FROM comment
WHERE parent is not NULL
GROUP BY parent
ORDER BY followup_num DESC
LIMIT 20
"""
comment_parent_df1 = spark.sql(sqlStatement)

In [30]:
# show top 10
comment_parent_df1.show(10)

+------------+--------+
|followup_num|  parent|
+------------+--------+
|         975|     363|
|         266| 7469115|
|         266| 9996333|
|         264| 9238839|
|         262| 9812245|
|         243| 7445761|
|         241|10152809|
|         239|  752262|
|         234| 9471287|
|         228| 9303396|
+------------+--------+
only showing top 10 rows



In [31]:
# ranking information
# what is the average ranking of those comments?

spark.sql("select min(ranking),max(ranking),avg(ranking) from comment").collect()

[Row(min(ranking)=-1051, max(ranking)=1131019295, avg(ranking)=58481.1891668736)]

In [32]:
# author/user's ranking information decending by avg ranking 
# who has the highest avg ranking?

sqlStatement = """SELECT avg(ranking) as avg_ranking,author,
count(id) as total_comment
FROM comment
GROUP BY author
ORDER BY avg(ranking) DESC
LIMIT 20
"""
comment_user_df1 = spark.sql(sqlStatement)

In [33]:
# show top 10
comment_user_df1.show(10)

+--------------------+------------+-------------+
|         avg_ranking|      author|total_comment|
+--------------------+------------+-------------+
|         2.0630034E7|AretNCarlsen|           56|
|1.5151532969696969E7|     dedalus|           66|
|         1.0360911E7|      pratim|            1|
|         1.0359641E7|    vmuhonen|            1|
|           1.03454E7|   scopesoft|            1|
|         1.0288613E7|  FlexMonkey|            1|
|         1.0277638E7|   Jimbobian|            1|
|         1.0275866E7| waynebeaton|            1|
|         1.0250132E7| anonttttttt|            1|
|         1.0230206E7|      bduhan|            1|
+--------------------+------------+-------------+
only showing top 10 rows



In [34]:
from pyspark.sql.functions import split

In [35]:
clean_comment_df.createOrReplaceTempView('tmp')

In [36]:
test = spark.sql('select * from tmp limit 200')

In [37]:
test.show()

+--------+-------------------+--------------------+--------+---------------+-------+
|      id|            time_ts|                text|  parent|         author|ranking|
+--------+-------------------+--------------------+--------+---------------+-------+
|  803232|2009-09-03 19:09:48|I've been coding ...|  802700|    heckacopter|      5|
| 8147414|2014-08-07 12:53:18|I think of Snowde...| 8146987|    justnotsure|      5|
|10331827|2015-10-05 13:59:13|Actually has led ...|10331623|    littletimmy|      5|
| 3058176|2011-09-30 19:04:37|Interesting siden...| 3054310|    underdesign|      5|
| 2582615|2011-05-25 02:27:49|Silly geeks.  A w...| 2582002|   originalgeek|      5|
| 9335083|2015-04-07 16:19:46|This.  I also lik...| 9331453|   pauldirac137|      5|
| 7349363|2014-03-05 19:48:55|These are just #h...| 7332992|  hipsterduuche|      5|
| 2769120|2011-07-15 21:11:01|Time to kiss Arri...| 2769046|  immortalbeast|      5|
| 6867776|2013-12-07 21:32:36|my classmate&#x27...| 6867543|  whi

In [42]:
test2 = test.withColumn("split", split("text", "\s+"))

In [43]:
test2.show()

+--------+-------------------+--------------------+--------+------+-------+--------------------+
|      id|            time_ts|                text|  parent|author|ranking|               split|
+--------+-------------------+--------------------+--------+------+-------+--------------------+
| 2701393|2011-06-27 14:28:01|And the glazier w...| 2701243|    5l|      0|[And, the, glazie...|
| 5811403|2013-06-03 04:34:08|Does canada have ...| 5804452|    99|      0|[Does, canada, ha...|
|   21623|2007-05-12 17:53:20|"Speaking of Rail...|   21611|    AF|      0|["Speaking, of, R...|
|10159727|2015-09-02 15:09:34|Humans and large ...|10159396|    EA|      0|[Humans, and, lar...|
| 2988424|2011-09-12 18:53:00|I must say I reac...| 2988179|    Iv|      0|[I, must, say, I,...|
| 3867418|2012-04-20 11:39:44|&#62; There's a w...| 3867404|    Iv|      0|[&#62;, There's, ...|
| 3925617|2012-05-03 20:26:05|I'm also in this ...| 3924840|    Iv|      0|[I'm, also, in, t...|
| 8409259|2014-10-04 11:20:47|

In [56]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [58]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenized = tokenizer.transform(test)
countTokens = udf(lambda words: len(words), IntegerType())
tokenized.select("text", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [89]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="words", pattern="\s+")
regexTokenized = regexTokenizer.transform(test)
regexTokenized.select("text", "words").show()

+--------------------+--------------------+
|                text|               words|
+--------------------+--------------------+
|My computing expe...|[my, computing, e...|
|Does anybody know...|[does, anybody, k...|
|Macbook Air (Or a...|[macbook, air, (o...|
|   awesome upgrade..|[awesome, upgrade..]|
|      Atlas Shrugged|   [atlas, shrugged]|
|Has anyone here m...|[has, anyone, her...|
|I'm eagerly waiti...|[i'm, eagerly, wa...|
|I always got jobs...|[i, always, got, ...|
|Trello and Everno...|[trello, and, eve...|
|I was waiting for...|[i, was, waiting,...|
|Microsoft is in t...|[microsoft, is, i...|
|He's out there so...|[he's, out, there...|
|I had the same fe...|[i, had, the, sam...|
|"Found this one v...|["found, this, on...|
|Wow. I didn't thi...|[wow., i, didn't,...|
|Just remember tha...|[just, remember, ...|
|I don't understan...|[i, don't, unders...|
|Looks like Spiege...|[looks, like, spi...|
|Sounds like a mas...|[sounds, like, a,...|
|Good idea but I h...|[good, ide

In [84]:
tokenized.show(5)

+-------+-------------------+--------------------+-------+---------+-------+--------------------+
|     id|            time_ts|                text| parent|   author|ranking|               words|
+-------+-------------------+--------------------+-------+---------+-------+--------------------+
|1876849|2010-11-06 13:39:56|minimalists don't...|1876724| thedeuce|      1|[minimalists, don...|
|1194782|2010-03-16 02:41:25|What if the write...|1194331| tphyahoo|      1|[what, if, the, w...|
| 415216|2008-12-31 06:32:21|you've missed the...| 415156| unrealwh|      1|[you've, missed, ...|
|8180803|2014-08-15 02:30:39|I, however, would...|8179627|Chronic29|      1|[i,, however,, wo...|
|5736766|2013-05-20 09:49:59|Thanks for this. ...|5736640|Gallefray|      1|[thanks, for, thi...|
+-------+-------------------+--------------------+-------+---------+-------+--------------------+
only showing top 5 rows



In [52]:
from pyspark.ml.feature import StopWordsRemover

In [61]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

In [66]:
filtered = remover.transform(tokenized)

In [90]:
filtered2 = remover.transform(regexTokenized)

In [67]:
filtered.show()

+--------+-------------------+--------------------+--------+---------------+-------+--------------------+--------------------+
|      id|            time_ts|                text|  parent|         author|ranking|               words|            filtered|
+--------+-------------------+--------------------+--------+---------------+-------+--------------------+--------------------+
|  803232|2009-09-03 19:09:48|I've been coding ...|  802700|    heckacopter|      5|[i've, been, codi...|[coding, almost, ...|
| 8147414|2014-08-07 12:53:18|I think of Snowde...| 8146987|    justnotsure|      5|[i, think, of, sn...|[think, snowden, ...|
|10331827|2015-10-05 13:59:13|Actually has led ...|10331623|    littletimmy|      5|[actually, has, l...|[actually, led, s...|
| 3058176|2011-09-30 19:04:37|Interesting siden...| 3054310|    underdesign|      5|[interesting, sid...|[interesting, sid...|
| 2582615|2011-05-25 02:27:49|Silly geeks.  A w...| 2582002|   originalgeek|      5|[silly, geeks., ,...|[silly

In [76]:
import pyspark.sql.functions as f

In [75]:
word_count = filtered.withColumn('word', f.explode(f.col('filtered')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show()

+---------+-----+
|     word|count|
+---------+-----+
|         |  119|
|   people|   50|
|     like|   40|
|    think|   30|
|      one|   25|
|     much|   24|
|      way|   24|
|     make|   23|
|      get|   23|
|   really|   22|
|      use|   19|
|something|   19|
|     good|   17|
|     know|   17|
|     also|   17|
|   things|   17|
|     need|   16|
|   better|   16|
|     even|   16|
|     want|   15|
+---------+-----+
only showing top 20 rows



In [91]:
word_count2 = filtered2.withColumn('word', f.explode(f.col('filtered')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show()

+---------+-----+
|     word|count|
+---------+-----+
|   people|   50|
|     like|   40|
|    think|   30|
|      one|   25|
|      way|   24|
|     much|   24|
|      get|   23|
|     make|   23|
|   really|   22|
|something|   19|
|      use|   19|
|     good|   17|
|     know|   17|
|   things|   17|
|     also|   17|
|     even|   16|
|     need|   16|
|   better|   16|
|     want|   15|
|   pretty|   15|
+---------+-----+
only showing top 20 rows



In [63]:
#下面是真的可能没啥用

In [48]:
tokenizer = Tokenizer(inputCol="text", outputCol="words") 
wordsData = tokenizer.transform(test)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)  


In [51]:
idf = IDF(inputCol="rawFeatures", outputCol="features")  
idfModel = idf.fit(featurizedData)  
rescaledData = idfModel.transform(featurizedData)  
for features_label in rescaledData.select("features", "id").take(3):  
    print(features_label)  

Row(features=SparseVector(20, {0: 0.4281, 2: 1.4164, 7: 1.3178, 8: 1.0005, 10: 0.8186, 12: 0.5759, 13: 0.4055, 15: 0.6028, 17: 0.4911, 18: 0.5075}), id=2701393)
Row(features=SparseVector(20, {1: 0.4993, 3: 1.015, 4: 1.1518, 5: 0.475, 6: 0.6399, 7: 0.6589, 8: 0.667, 10: 0.5457, 11: 0.6783, 14: 0.5671, 16: 0.4435, 18: 0.5075, 19: 0.5326}), id=5811403)
Row(features=SparseVector(20, {0: 1.7124, 1: 2.9957, 2: 0.7082, 3: 2.0301, 4: 3.4555, 5: 7.5999, 6: 3.1993, 7: 3.2946, 8: 4.0019, 9: 1.0652, 10: 3.2744, 11: 4.07, 12: 1.7278, 13: 2.0273, 14: 3.4026, 15: 2.4113, 16: 2.2175, 17: 1.4734, 18: 2.5376, 19: 1.5979}), id=21623)


In [ ]:
# 正则
# split
